In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("./examples/data/Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
features=df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
features=features.to_numpy()
features = features.T

In [4]:
features.shape

(4, 150)

In [5]:
labels=pd.get_dummies(df["Species"]).to_numpy()
labels=labels.T

In [6]:
def SoftMax(predictions:np.array):
    predictions = np.exp(predictions)
    return predictions/predictions.sum(axis=0,keepdims=True)

def Sigmoid(sumatory,derivative=False):
    if derivative==False:
        return 1/(1+np.exp(-sumatory))
    else :
        return np.exp(-sumatory)/((1+ np.exp(-sumatory)))**2

def CategoricalEntropy(labels:np.array,predictions:np.array):
    errors = -1*labels*np.log(predictions)
    return errors.sum()


def Accurcy(preds,labels):

    preds= preds.T
    labels= labels.T
    correct = 0 

    for pred,lab in zip(preds,labels):
        pred_max_index = np.where(pred==np.max(pred))[0][0]
        lab_max_index = np.where(lab==np.max(lab))[0][0]
        
        if pred_max_index == lab_max_index:
            correct+=1

    return correct/len(preds)

In [7]:
class LayerDense:
    def __init__(self,n_neurones:int,n_features:int,activation_function:str):
        self.n_neurones = n_neurones
        self.n_features = n_features
        self.weights=np.random.randn(self.n_neurones,self.n_features)**2
        self.bias=np.random.randn(self.n_neurones,1)**2
        self.z:np.array = None
        self.activation_function = activation_function
        self.activation=None

    def foward(self,features):
        self.z = self.weights@features + self.bias
        self.activation  = self.activation_function(self.z)
        return self.activation


    def backward_final_layer(self,lr,Delta_l,activation):

        DcDw=Delta_l@activation.T
        DcDb=Delta_l.sum(axis=1).reshape(-1,1)

        self.weights = self.weights - lr*DcDw
        self.bias = self.bias - lr*DcDb

        return Delta_l

    def backward(self,lr,Delta_l,Activation_l_1,Weights_l):

        Delta_l = Weights_l.T@Delta_l*self.activation_function(self.z,True)

        DcDw=Delta_l@Activation_l_1.T
        DcDb=Delta_l.sum(axis=1).reshape(-1,1)

        self.weights = self.weights - lr*DcDw
        self.bias = self.bias - lr*DcDb

        return Delta_l

    def backward_first_layer(self,lr,Delta_l,Activation_l_1,Weights_l):

        Delta_l = Weights_l.T@Delta_l*self.activation_function(self.z,True)
        
        DcDw=Delta_l@Activation_l_1.T
        DcDb=Delta_l.sum(axis=1).reshape(-1,1)

        self.weights = self.weights - lr*DcDw
        self.bias = self.bias - lr*DcDb


In [8]:
class NN:
    def __init__(self,loss_function,metric,learning_rate):
        self.layers: dict[int,LayerDense]={}
        self.n_layers = 0
        self.loss_function = loss_function
        self.learning_rate = learning_rate
        self.metric = metric

    def add_layer(self,layer):
        self.n_layers += 1
        self.layers[self.n_layers] = layer 

    def foward(self,features):
        for key in self.layers.keys():
            features = self.layers[key].foward(features)
        return features

    def bacward(self,activation,labels,features):
        DcDz = activation - labels
        for i,layer in reversed(model.layers.items()):
            if i == self.n_layers:
                Activation_l_1 = self.layers[i-1].activation
                DcDz = layer.backward_final_layer(self.learning_rate,DcDz,Activation_l_1)
            else:
                Weights_l = self.layers[i+1].weights
                if i == 1:
                    Activation_l_1 = features
                    layer.backward_first_layer(self.learning_rate,DcDz,Activation_l_1,Weights_l)
                else:
                    Activation_l_1 = self.layers[i-1].activation
                    DcDz = layer.backward(self.learning_rate,DcDz,Activation_l_1,Weights_l)   

In [32]:
model = NN(CategoricalEntropy,Accurcy,0.001)
model.add_layer(LayerDense(4,4,Sigmoid))
model.add_layer(LayerDense(3,4,SoftMax))

iters = 1000
steps = int(iters/10)
for i  in range(1,iters):
    activation = model.foward(features)
    model.bacward(activation,labels,features)
    if  i%steps == 0:
        error=CategoricalEntropy(labels,activation)
        accuracy=Accurcy(activation,labels)
        print(f"Iter: {i}\tError: {error:.2f}\tAccuracy: {accuracy*100:.3f}%")

Iter: 100	Error: 107.26	Accuracy: 68.667%
Iter: 200	Error: 87.60	Accuracy: 70.000%
Iter: 300	Error: 80.50	Accuracy: 74.000%
Iter: 400	Error: 76.62	Accuracy: 84.000%
Iter: 500	Error: 73.34	Accuracy: 87.333%
Iter: 600	Error: 69.60	Accuracy: 89.333%
Iter: 700	Error: 65.52	Accuracy: 90.667%
Iter: 800	Error: 61.57	Accuracy: 92.667%
Iter: 900	Error: 58.00	Accuracy: 94.000%
